Hello, bevor ihr alle Zellen ausführt bitte einmal schauen dass die vier Corpora Dateien, die ich mit euch geteilt habe mit einem Shortcut auf eurem Drive verbunden sind, sonst können die nicht gefunden werden. Danke:) 

Rest sollte einfach durchlaufen, die ganze Struktur und alles wird automatisch erstellt.
Im Optimalfall denke ich etwa 45 Minuten pro Übersetzung ist aber sehr abhängig von den Ressourcen, kann auch mal 1.5h dauern.

In [ ]:
# Installs
!pip install transformers
!pip install sentencepiece
!pip install sacremoses

In [6]:
# Imports
#from transformers import pipeline
from google.colab import drive
drive.mount('/content/drive')
import os
from os import path
import time
#import torch

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Make sure GPU is available
assert torch.cuda.is_available() == True and torch.cuda.device_count() == 1 and torch.cuda.current_device() == 0

In [8]:
# Environment and other stuff
!if [ ! -d "/content/drive/MyDrive/mlt_final_project"]; then mkdir "/content/drive/MyDrive/mlt_final_project"; else echo "Directory /content/drive/mlt_final_project already exists"; fi
proj_path = "/content/drive/MyDrive/mlt_final_project"
os.environ["proj_path"] = proj_path
!if [ ! -d "${proj_path}/corpora"]; then mkdir -p "${proj_path}/corpora/de-en"; else echo "Directory ${proj_path}/corpora/de-en already exists"; fi
!if [ ! -d "${proj_path}/corpora"]; then mkdir -p "${proj_path}/corpora/de-es"; else echo "Directory ${proj_path}/corpora/de-es already exists"; fi

/bin/bash: line 0: [: missing `]'
Directory /content/drive/mlt_final_project already exists
/bin/bash: line 0: [: missing `]'
Directory $/content/drive/MyDrive/mlt_final_project/corpora/de-en already exists
/bin/bash: line 0: [: missing `]'
Directory $/content/drive/MyDrive/mlt_final_project/corpora/de-es already exists


In [ ]:
!touch "${pp_deen}/pp_deen.de"
!touch "${pp_deen}/pp_deen.en"
!touch "${pp_dees}/pp_dees.de"
!touch "${pp_dees}/pp_dees.es"

In [ ]:
# Create some more important directories and files
pp_deen_path = f'{proj_path}/preprocessed/de-en'
pp_dees_path = f'{proj_path}/preprocessed/de-es'
os.environ["pp_deen"] = pp_deen_path
os.environ["pp_dees"] = pp_dees_path
!if [! -d "${pp_deen}"]; then mkdir -p "${pp_deen}"; else echo "Directory ${pp_deen} already exists"; fi
!if [! -d "${pp_dees}"]; then mkdir -p "${pp_dees}"; else echo "Directory ${pp_dees} already exists"; fi

# create files for preprocessed data
!if [! -L "${pp_deen}/pp_deen.de"]; then touch "${pp_deen}/pp_deen.de"; else echo "File ${pp_deen}/pp_deen.de already exists"; fi
!if [! -L "${pp_deen}/pp_deen.en"]; then touch "${pp_deen}/pp_deen.en"; else echo "File ${pp_deen}/pp_deen.en already exists"; fi
!if [! -L "${pp_dees}/pp_dees.de"]; then touch "${pp_dees}/pp_dees.de"; else echo "File ${pp_dees}/pp_dees.de already exists"; fi
!if [! -L "${pp_dees}/pp_dees.es"]; then touch "${pp_dees}/pp_dees.es"; else echo "File ${pp_dees}/pp_dees.es already exists"; fi

# copy data from Shared to local to work with it
!find "/content/drive/MyDrive" -type f -name '*europarl-v7' | xargs cp -t "${proj_path}/corpora/de-en"
!find "/content/drive/MyDrive" -type f -name '*Europarl.de-es*' | xargs cp -t "${proj_path}/corpora/de-es"

/bin/bash: [!: command not found
Directory /content/drive/MyDrive/mlt_final_project/preprocessed/de-en already exists
/bin/bash: [!: command not found
Directory /content/drive/MyDrive/mlt_final_project/preprocessed/de-es already exists
/bin/bash: [!: command not found
File /content/drive/MyDrive/mlt_final_project/preprocessed/de-en/pp_deen.de already exists
/bin/bash: [!: command not found
File /content/drive/MyDrive/mlt_final_project/preprocessed/de-en/pp_deen.en already exists
/bin/bash: [!: command not found
File /content/drive/MyDrive/mlt_final_project/preprocessed/de-es/pp_dees.de already exists
/bin/bash: [!: command not found
File /content/drive/MyDrive/mlt_final_project/preprocessed/de-es/pp_dees.es already exists
cp: failed to access '$/content/drive/MyDrive/mlt_final_project/corpora/de-en': No such file or directory
cp: failed to access '$/content/drive/MyDrive/mlt_final_project/corpora/de-es': No such file or directory


In [9]:
# Load data
data_path_de_en = path.join(proj_path, "corpora", "de-en")
data_path_de_es = path.join(proj_path, "corpora", "de-es")

file_path_de_en = path.join(data_path_de_en, "europarl-v7.de-en")
file_path_de_es = path.join(data_path_de_es, "Europarl.de-es")

def load_data_x(path_obj: str):
  if not path.isfile(path_obj):
    print(f'Object with path {path_obj} is not a file')
    return None
  try:
    with open(path_obj, "r") as f:
      data_obj = f.read()
      return data_obj
  except Exception as e:
    print(f'Failed to open file with error message: {str(e)}')


filename_deen_de = ''.join([file_path_de_en, ".de"])
filename_dees_de = ''.join([file_path_de_es, ".de"])

data_deen_de = load_data_x(filename_deen_de)
data_dees_de = load_data_x(filename_dees_de)

In [ ]:
filename_deen_en = ''.join([file_path_de_en, ".en"])
filename_dees_es = ''.join([file_path_de_es, ".es"])
data_deen_en = load_data_x(filename_deen_en)
data_dees_es = load_data_x(filename_dees_es)

In [3]:
def preprocess(num_of_lines: int, text: str):
  return text.split("\n")[:num_of_lines]

In [10]:
proj_path = "/content/drive/MyDrive/mlt_final_project"

filename_deen_en = ''.join([file_path_de_en, ".en"])
data_deen_en = load_data_x(filename_deen_en)
pp_deen_path = f'{proj_path}/preprocessed/de-en'

pp_en = preprocess(50000, data_deen_en)
save_write_file(pp_en, f'{pp_deen_path}/pp_deen.en_test') 

In [4]:
def save_write_file(data: list, file_path: str):
  with open(file_path, "w") as f:
    for line in data:
      f.write(line + "\n")

In [ ]:
save_write_file(pp_data[0], f'{pp_deen_path}/pp_deen.de') 
save_write_file(pp_data[1], f'{pp_deen_path}/pp_deen.en') 
save_write_file(pp_data[2], f'{pp_dees_path}/pp_dees.de')
save_write_file(pp_data[3], f'{pp_dees_path}/pp_dees.es')

In [ ]:
# Translate
pipe_en = pipeline(task="translation_de_to_en", model="Helsinki-NLP/opus-mt-de-en", device=0, batch_size=16)
pipe_es = pipeline(task="translation_de_to_es", model="Helsinki-NLP/opus-mt-de-es", device=0, batch_size=16)
pipe_ende = pipeline(task="translation_en_to_de", model="Helsinki-NLP/opus-mt-en-de", device=0, batch_size=16)
pipe_esde = pipeline(task="translation_es_to_de", model="Helsinki-NLP/opus-mt-es-de", device=0, batch_size=16)

def translate(lines: list, language: str):
  translated_lines = {}
  if language is "en":
    translated_lines = pipe_en(lines)
  elif language is "es":
    translated_lines = pipe_es(lines)
  return translated_lines


Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/284M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/778k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/750k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/290M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/799k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/809k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

In [ ]:
!if [! -d "${proj_path}/translated/de-en"]; then mkdir -p "${proj_path}/translated/de-en"; else echo "Directory ${proj_path}/translated/de-en already exists"; fi
!if [! -d "${proj_path}/translated/de-es"]; then mkdir -p "${proj_path}/translated/de-es"; else echo "Directory ${proj_path}/translated/de-es already exists"; fi
os.environ["translated_deen"] = f'{proj_path}/translated/de-en'
os.environ["translated_dees"] = f'{proj_path}/translated/de-es'
# !touch "${translated_deen}/translated.en"
# !touch "${translated_dees}/translated.es"
translated_deen_path = path.join(f'{proj_path}/translated/de-en', "translated.en")
translated_dees_path = path.join(f'{proj_path}/translated/de-es', "translated.es")

/bin/bash: [!: command not found
Directory /translated/de-en already exists
/bin/bash: [!: command not found
Directory /translated/de-es already exists


NameError: ignored

In [ ]:
def extract_val(dictionary: dict, key: str):
  return dictionary[key]


def extract_all(items: list, extract_key: str):
  extracted = []
  for item in items:
    extracted.append(extract_val(item, extract_key))
  return extracted

In [ ]:
# Translation to English
start_time_deen = time.time()
translated_deen_en = translate(pp_data[0], "en")
end_time_deen = time.time()
total_time_deen = end_time_deen - start_time_deen
print(total_time_deen)
trans_extr_en = extract_all(translated_deen_en, "translation_text")
save_write_file(trans_extr_en, translated_deen_path)

# Translation to Spanish
start_time_dees = time.time()
translated_dees_es = translate(pp_data[2], "es")
end_time_dees = time.time()
total_time_dees = end_time_dees - start_time_dees
print(total_time_dees)
trans_extr_es = extract_all(translated_dees_es, "translation_text")
save_write_file(trans_extr_es, translated_dees_path)

2514.6251604557037
2754.7833807468414


In [ ]:
# forward and backward translation functions

def backward_translate_en(lines):
  translations = {}
  back_start = time.time()
  translations = pipe_ende(lines)
  trans_extr = extract_all(translations, "translation_text")
  back_end = time.time()
  print(f'The english to german translation took {back_end-back_start} seconds.')
  return trans_extr


def forward_translate_en(lines):
  translations = {}
  forward_start = time.time()
  translations = pipe_en(lines)
  trans_extr = extract_all(translations, "translation_text")
  forward_end = time.time()
  print(f'The german to english translation took {forward_end-forward_start} seconds.')
  return trans_extr


def backward_translate_es(lines):
  translations = {}
  back_start = time.time()
  translations = pipe_esde(lines)
  trans_extr = extract_all(translations, "translation_text")
  back_end = time.time()
  print(f'The spanish to german translation took {back_end-back_start} seconds.')
  return trans_extr


def forward_translate_es(lines):
  translations = {}
  forward_start = time.time()
  translations = pipe_es(lines)
  trans_extr = extract_all(translations, "translation_text")
  forward_end = time.time()
  print(f'The german to spanish translation took {forward_end-forward_start} seconds.')
  return trans_extr

In [ ]:
# bidirectional translation functions

it_trl_path = "/content/drive/MyDrive/mlt_final_project/iterative_translation"

def bidirectional_translation_en(num_of_translations, lines):
  backward_trl = backward_translate_en(lines)
  # print(f'Saving to path {it_trl_path}/backward_trl_ende_0.txt')
  # print(f'Length of backward_trl is: {len(backward_trl)}')
  save_write_file(backward_trl, path.join(it_trl_path, f'backward_trl_ende_0.txt'))
  forward_trl = forward_translate_en(backward_trl)
  # print(f'Saving to path {it_trl_path}/forward_trl_deen_0.txt')
  # print(f'Length of forward_trl is: {len(forward_trl)}')
  save_write_file(forward_trl, path.join(it_trl_path, f'forward_trl_deen_0.txt'))
  
  for number in range(num_of_translations):
    backward_trl = backward_translate_en(forward_trl)
    # print(f'Saving to path {it_trl_path}/backward_trl_ende_{number+1}.txt')
    # print(f'Length of backward_trl is: {len(backward_trl)}')
    save_write_file(backward_trl, path.join(it_trl_path, f'backward_trl_ende_{number+1}.txt'))
    forward_trl = forward_translate_en(backward_trl)
    # print(f'Saving to path {it_trl_path}/forward_trl_deen_{number+1}.txt')
    # print(f'Length of forward_trl is: {len(forward_trl)}')
    save_write_file(forward_trl, path.join(it_trl_path, f'forward_trl_deen_{number+1}.txt'))


def bidirectional_translation_es(num_of_translations, lines):
  # backward_trl = backward_translate_es(lines)
  backward_trl = []
  # print(f'Saving to path {it_trl_path}/backward_trl_esde_0.txt')
  # print(f'First line of backward_trl is: {backward_trl[0]}')
  # save_write_file(backward_trl, path.join(it_trl_path, f'backward_trl_esde_2.txt'))
  forward_trl = forward_translate_es(lines)
  # print(f'Saving to path {it_trl_path}/forward_trl_dees_0.txt')
  # print(f'First line of forward_trl is: {forward_trl[0]}')
  save_write_file(forward_trl, path.join(it_trl_path, f'forward_trl_dees_2.txt'))

  for number in range(num_of_translations):
    backward_trl = backward_translate_es(forward_trl)
    # print(f'Saving to path {it_trl_path}/backward_trl_esde_{number+1}.txt')
    # print(f'First line of backward_trl is: {backward_trl[0]}')
    save_write_file(backward_trl, path.join(it_trl_path, f'backward_trl_esde_{number+3}.txt'))
    forward_trl = forward_translate_es(backward_trl)
    # print(f'Saving to path {it_trl_path}/forward_trl_dees_{number+1}.txt')
    # print(f'First line of forward_trl is: {forward_trl[0]}')
    save_write_file(forward_trl, path.join(it_trl_path, f'forward_trl_dees_{number+3}.txt'))

In [ ]:
def load_data(file_path):
  data = []
  with open(file_path, "r") as f:
    data = f.readlines()
  return data


def rm_all_newlines(*sentences):
  return [s.replace("\n", "") for s in sentences]

In [ ]:
translated_path = "/content/drive/MyDrive/mlt_final_project/translated"
# translated_en_path = path.join(translated_path, "de-en", "translated.en")
translated_es_path = path.join(translated_path, "de-es", "translated.es")
# trl_data_en = load_data(translated_en_path)
# trl_data_es = load_data(translated_es_path)
translated_esde_path = path.join(it_trl_path, "backward_trl_esde_2.txt")
trl_data_esde = load_data(translated_esde_path)
# stripped_trl_en = rm_all_newlines(*trl_data_en)
#stripped_trl_es = rm_all_newlines(*trl_data_es)
stripped_trl_esde = rm_all_newlines(*trl_data_esde)

In [ ]:
# iterative translation
# bidirectional_translation_en(2, stripped_trl_en)
bidirectional_translation_es(3, stripped_trl_esde)

Your input_length: 510 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 511 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


The german to spanish translation took 2596.875186920166 seconds.


Your input_length: 511 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 511 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 511 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


The spanish to german translation took 2515.908399581909 seconds.


Your input_length: 510 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 511 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


The german to spanish translation took 2577.40828704834 seconds.


Your input_length: 511 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 511 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


In [ ]:
aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa

In [ ]:
import json
 
# Opening JSON file
data_frame = pd.dataframe()
text_list = data_frame['Text'].to_list()
with open('data.json') as json_file:
    data = json.load(json_file)
    intermediate_dict = {}
    keys = data.keys()
    values = data.values()
    general = np.unique(keys+values)
    lemma_list = [for item in text_list True if item in general else False] 
data_frame['relevant_lemma'] = lemma_list

In [ ]:
# shannon entropy
def shannon_entropy(dataframe)
  if 'relevant_lemma' in dataframe.columns:
    subframe = dataframe.loc[dataframe['relevant_lemma'] == True]
  else:
    subframe = dataframe
  lemma_frames = dataframe.groupby('Lemma')
  entropy_values=[]
  for lemma, frame in lemma_frames:
    words, word_counts = np.unique(frame['Text'].to_numpy(), return_counts=True)
    normalized_word_counts = word_counts / np.sum(word_counts)
    entropy = -np.sum(normalized_word_counts * np.log(normalized_word_counts))
    entropy_values.append(entropy)
  return np.mean(entropy_values)

In [ ]:
# Simpson's diversity
def simpson_diversity(dataframe)
  if 'relevant_lemma' in dataframe.columns:
    subframe = dataframe.loc[dataframe['relevant_lemma'] == True]
  else:
    subframe = dataframe
  lemma_frames = subframe.groupby('Lemma')
  diversity_values=[]
  for lemma, frame in lemma_frames:
    words, word_counts = np.unique(frame['Text'].to_numpy(), return_counts=True)
    normalized_word_counts = word_counts / np.sum(word_counts)
    diversity = 1/np.sum(normalized_word_counts**2)
    diversity_values.append(diversity)
  return np.mean(diversity_values)

In [ ]:
# Lexical Frequency Profile
def get_lfp(data_frame, index_dict=None, return_index_dict=False):

    words, word_counts = np.unique(data_frame['Text'].to_numpy(), return_counts=True)
    if index_dict:
      new_word_indices = []
      restructured_indices = []
      for word in words:
        if word in index_dict.keys():
          restructured_indices.append(index_dict[word])
          new_word_indices.append(False)
        else:
          new_word_indices.append(True)

      sorted_word_counts = word_counts[restructured_indices]          
      new_words = words[new_word_indices]
      new_word_counts = word_counts[new_word_indices]
      sorted_word_counts = np.append(sorted_word_counts, new_word_counts)
      denominator = np.sum(sorted_word_counts)
      first_thousand_percentage = np.sum(sorted_word_counts[:1000])/denominator
      second_thousand_percentage = np.sum(sorted_word_counts[1000:2000])/denominator
      rest_percentage = np.sum(sorted_word_counts[2000:])/denominator
      new_word_percentage = np.sum(new_word_counts)/denominator

    
    else:
      sorted_indices = np.argsort(word_counts)
    # potential to get the actual most used words
      sorted_words = words[sorted_indices]
      sorted_word_counts = word_counts[sorted_indices]
      denominator = np.sum(sorted_word_counts)
      first_thousand_percentage = np.sum(sorted_word_counts[:1000])/denominator
      second_thousand_percentage = np.sum(sorted_word_counts[1000:2000])/denominator
      rest_percentage = np.sum(sorted_word_counts[2000:])/denominator
      index_dict = None
      if return_index_dict:
        for word, index in zip(sorted_words, sorted_indices)
          index_dict[word] = index
    return [first_thousand_percentage, second_thousand_percentage, rest_percentage], index_dict

In [ ]:
def create_histogram(csv_path, save_path=None):
    patient_data_frame = pd.read_csv(csv_path)
    age_array = np.zeros((4, patient_data_frame['Patient Age'].max()+1), dtype=np.int16)
    #print(patient_data_frame['Patient Age'].max(), patient_data_frame['Patient Age'].min())
    #print(age_array.shape)
    #print(patient_data_frame.loc[patient_data_frame['Patient Age']==0])
    male_patients = patient_data_frame.loc[patient_data_frame['Patient Gender'] =='M'].sort_values('Image Index')
    female_patients = patient_data_frame.loc[patient_data_frame['Patient Gender']=='F'].sort_values('Image Index')
    male_patients_ap = male_patients.loc[patient_data_frame['View Position'] =='AP'].sort_values('Image Index')
    male_patients_pa = male_patients.loc[patient_data_frame['View Position'] =='PA'].sort_values('Image Index')
    female_patients_ap = female_patients.loc[patient_data_frame['View Position'] =='AP'].sort_values('Image Index')
    female_patients_pa = female_patients.loc[patient_data_frame['View Position'] =='PA'].sort_values('Image Index')
    overall_age, overall_counts = np.unique(patient_data_frame['Patient Age'].to_numpy(), return_counts=True)
    map_age, map_counts = np.unique(male_patients_ap['Patient Age'].to_numpy(), return_counts=True)
    mpa_age, mpa_counts = np.unique(male_patients_pa['Patient Age'].to_numpy(), return_counts=True)
    apf_age, apf_counts = np.unique(female_patients_ap['Patient Age'].to_numpy(), return_counts=True)
    paf_age, paf_counts = np.unique(female_patients_pa['Patient Age'].to_numpy(), return_counts=True)
    #print(mpa_age, mpa_counts,apf_age, apf_counts)
    #print(map_age, map_counts,paf_age, paf_counts)
    age_array[0, mpa_age]=age_array[0, mpa_age] + mpa_counts
    age_array[1, map_age]=age_array[1, map_age] + map_counts
    age_array[2, paf_age]=age_array[2, paf_age] + paf_counts
    age_array[3, apf_age]=age_array[3, apf_age] + apf_counts
    age_range = np.arange(len(age_array[0]))

    plt.figure(figsize=(20,5))
    plt.bar(age_range, age_array[0],label='male PA', color="tab:red")

    # the second one is special to create stacked bar plots
    plt.bar(age_range, age_array[1], bottom=age_array[0],label='male AP', color="tab:orange")
    plt.bar(age_range, age_array[2], bottom=np.sum(age_array[:2], axis=0),label='female PA', color="tab:green")
    plt.bar(age_range, age_array[3], bottom=np.sum(age_array[:3], axis=0),label='female AP', color="tab:blue")
    plt.legend(loc="best")
    plt.xlabel('Age')  
    plt.ylabel('Instances')

    BIGGER_SIZE = 20
    plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
    plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
    plt.tight_layout()
    handles, labels = plt.gca().get_legend_handles_labels()

    order = [3,2,1,0]
    plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order])
    if save_path:
      plt.savefig(save_path)

    plt.show()  
    plt.figure(figsize=(20,5))
    overall_age, overall_counts = np.unique(patient_data_frame['Patient Age'].to_numpy(), return_counts=True)
    plt.bar(overall_age, overall_counts, color="tab:purple",  width=0.8)
    plt.show()
    #print(age_array)